In [1]:
# **1: Data Loading And Processing**

import os
import time

import cv2
import numpy as np

# Define the path to the folder containing the spectrogram images
data_dir = r'path to dataset folder'

# Define the classes (folder names)
classes = os.listdir(data_dir)
num_classes = len(classes)

# Define the desired width and height for resizing
desired_width = 224
desired_height = 224

# Initialize lists to store spectrogram images and their corresponding labels
images_list = []
labels_list = []

# Iterate through each class folder
for class_idx, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    # Iterate through each spectrogram image in the class folder
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        # Read the image using OpenCV
        img = cv2.imread(img_path)
        # Resize the image to the desired dimensions without preserving the aspect ratio
        img = cv2.resize(img, (desired_width, desired_height))
        # Convert the image to float32 and normalize pixel values to [0, 1]
        img = img.astype(np.float32) / 255.0
        # Append the preprocessed image and its label to the lists
        images_list.append(img)
        labels_list.append(class_idx)  # Use class index as label

# Convert the lists to numpy arrays
images = np.array(images_list)
labels = np.array(labels_list)

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Shuffle the data
indices = np.arange(images.shape[0])
np.random.shuffle(indices)
images = images[indices]
labels = labels[indices]

# Split the data into training, validation, and test sets (e.g., 80-10-10 split)
train_split = int(0.8 * len(images))
val_split = int(0.10 * len(images))
x_train, y_train = images[:train_split], labels[:train_split]
x_val, y_val = images[train_split:train_split + val_split], labels[train_split:train_split + val_split]
x_test, y_test = images[train_split + val_split:], labels[train_split + val_split:]

In [12]:
import tensorflow as tf
import tensorflow.keras.backend as K

In [ ]:
"""# **2:Siamese Network Architecture:**"""

from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, concatenate
from keras.models import Model

# Define input shape
input_shape = (desired_height, desired_width, 3)  # Assuming images are RGB

# Define Siamese CNN architecture
input_left = Input(shape=input_shape)
input_right = Input(shape=input_shape)

# Shared convolutional layers
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')
pool1 = MaxPooling2D(pool_size=(2, 2))
conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu')
pool2 = MaxPooling2D(pool_size=(2, 2))
flatten = Flatten()

# Process left input
x1 = conv1(input_left)
x1 = pool1(x1)
x1 = conv2(x1)
x1 = pool2(x1)
x1 = flatten(x1)

# Process right input
x2 = conv1(input_right)
x2 = pool1(x2)
x2 = conv2(x2)
x2 = pool2(x2)
x2 = flatten(x2)

# Concatenate the processed inputs
concatenated = concatenate([x1, x2])

# Dense layers for classification
dense1 = Dense(128, activation='relu')(concatenated)
output = Dense(num_classes, activation='softmax')(dense1)

# Create Siamese CNN model
siamese_model = Model(inputs=[input_left, input_right], outputs=output)

# Compile the model
siamese_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
siamese_model.summary()

In [ ]:
"""# **3:Training**"""

# Define number of epochs and batch size
epochs = 10
batch_size = 16

# Train the model
history = siamese_model.fit(
    [x_train, x_train],  # Pass the left and right images as inputs
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=([x_val, x_val], y_val),  # Pass validation data
    verbose=1
)

In [ ]:
"""# **4:Evaluation**"""

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
)

# Use the trained model to predict classes for the test set
y_pred = siamese_model.predict([x_test, x_test])
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')
f2 = fbeta_score(y_test, y_pred_classes, average='weighted', beta=2)

print("Accuracy: %.5f", accuracy)
print("Precision: %.5f", precision)
print("Recall: %.5f", recall)
print("F1-score: %.5f", f1)
print("F2-score: %.5f", f2)

In [ ]:
"""# **5:Confusion Matrix**"""

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Assuming y_test and y_pred_classes are defined
# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Generate normalized confusion matrix
conf_matrix_normalized = confusion_matrix(y_test, y_pred_classes, normalize='true')

# Set up subplots to display both matrices side by side
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot normal confusion matrix
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes, ax=axes[0])
axes[0].set_title('Confusion Matrix (Counts)')
axes[0].set_xlabel('Predicted Labels')
axes[0].set_ylabel('True Labels')

# Plot normalized confusion matrix
sns.heatmap(conf_matrix_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=classes, yticklabels=classes, ax=axes[1])
axes[1].set_title('Normalized Confusion Matrix (Proportions)')
axes[1].set_xlabel('Predicted Labels')
axes[1].set_ylabel('True Labels')

# Adjust layout and display the plot
plt.tight_layout()
plt.savefig("conf.png", dpi=650)
plt.show()


In [ ]:
"""# **6:Save the Model**"""

# Define the path where you want to save the model
model_path = 'path to saved model'

# Save the model
siamese_model.save(model_path)
print("Model saved successfully at:", model_path)

"""# **7:For loading the model**"""

# from keras.models import load_model
# model_path = r'C:\Users\AKmoh\OneDrive\Desktop\hari proj never delete\ecgmini\expnew\latestmodel2.keras'
# # Load the saved model
# loaded_model = load_model(model_path)

# Use the loaded model for inference or further training

In [ ]:
"""# **8:PREDICTION**"""

import cv2
import numpy as np

# Load the external spectrogram
external_spectrogram_path = 'path to test data'
external_spectrogram = cv2.imread(external_spectrogram_path)

# Preprocess the external spectrogram
desired_width = 224
desired_height = 224
external_spectrogram_resized = cv2.resize(external_spectrogram, (desired_width, desired_height))
external_spectrogram_normalized = external_spectrogram_resized.astype(np.float32) / 255.0

# Reshape the input to match model's input shape (add batch dimension)
input_external = np.expand_dims(external_spectrogram_normalized, axis=0)

# Use the trained model to predict classes for the external spectrogram
predictions = siamese_model.predict([input_external, input_external])

# Convert predictions to class labels
predicted_class_index = np.argmax(predictions)

# Print the predicted class label
print("Predicted class:", classes[predicted_class_index])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# Assuming y_test and y_pred_proba are defined, and num_classes is the number of classes
# Binarize the output for multi-class ROC
y_pred_proba = siamese_model.predict([x_test, x_test])
y_test_bin = label_binarize(y_test, classes=np.arange(num_classes))

# Compute micro-average ROC curve and AUC
fpr_micro, tpr_micro, _ = roc_curve(y_test_bin.ravel(), y_pred_proba.ravel())
roc_auc_micro = auc(fpr_micro, tpr_micro)

# Plot micro-average ROC curve
plt.figure(figsize=(10, 8))
plt.plot(fpr_micro, tpr_micro, color='blue', lw=2,
         label='Micro-average ROC curve (AUC = %0.2f)' % roc_auc_micro)

# Plot diagonal line for random guessing
plt.plot([0, 1], [0, 1], 'k--')

# Set limits and labels
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Micro-average Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.savefig('roc.png', dpi= 650)
plt.show()

# Print overall micro-average AUC
# print(f'Micro-average AUC: {roc_auc_micro:.2f}')


In [ ]:
"""# **10: Accuracy Curve**"""

import matplotlib.pyplot as plt

# Extract training and validation accuracy from history
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot accuracy curve
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.title('Accuracy Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('acc.png', dpi=650)
plt.show()

In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Define the path to the new dataset directory
data_dir = 'path to dataset folder'

# Load the dataset from the directory
# Assuming the directory has subdirectories representing each class
def load_dataset(data_dir, img_size=(224, 224)):
    class_names = os.listdir(data_dir)  # List of class names
    images = []
    labels = []
    
    # Iterate through each class directory
    for class_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)  # Read image
            img = cv2.resize(img, img_size)  # Resize image
            img = img.astype(np.float32) / 255.0  # Normalize to [0, 1]
            images.append(img)
            labels.append(class_idx)  # Assign class index as label

    # Convert to numpy arrays
    images = np.array(images)
    labels = np.array(labels)
    
    # Shuffle the dataset
    indices = np.arange(images.shape[0])
    np.random.shuffle(indices)
    images = images[indices]
    labels = labels[indices]

    return images, labels, class_names

# Load the dataset
x_data, y_data, classes = load_dataset(data_dir)

# Split the data into training, validation, and test sets (e.g., 80-10-10 split)
train_split = int(0.8 * len(x_data))
val_split = int(0.10 * len(x_data))
x_train, y_train = x_data[:train_split], y_data[:train_split]
x_val, y_val = x_data[train_split:train_split + val_split], y_data[train_split:train_split + val_split]
x_test, y_test = x_data[train_split + val_split:], y_data[train_split + val_split:]

# Load the saved Siamese model
model_path = 'path to saved model'
siamese_model = tf.keras.models.load_model(model_path)

# Predict on the test set
y_pred_proba = siamese_model.predict([x_test, x_test])
y_pred_classes = np.argmax(y_pred_proba, axis=1)

# Compute overall metrics
overall_accuracy = accuracy_score(y_test, y_pred_classes)
overall_f1 = f1_score(y_test, y_pred_classes, average='weighted')
overall_precision = precision_score(y_test, y_pred_classes, average='weighted')
overall_recall = recall_score(y_test, y_pred_classes, average='weighted')

# Print overall metrics
print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print(f"Overall F1-Score: {overall_f1:.4f}\n")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Initialize lists to store metrics for each class
class_wise_accuracy = []
class_wise_f1 = []
class_wise_precision = []
class_wise_recall = []

# Calculate metrics for each class
for i in range(len(classes)):
    true_positives = conf_matrix[i, i]
    false_positives = np.sum(conf_matrix[:, i]) - true_positives
    false_negatives = np.sum(conf_matrix[i, :]) - true_positives
    true_negatives = np.sum(conf_matrix) - (true_positives + false_positives + false_negatives)
    
    # Class-wise accuracy
    class_accuracy = (true_positives + true_negatives) / np.sum(conf_matrix)
    class_wise_accuracy.append(class_accuracy)
    
    # Class-wise precision, recall, and F1 score
    class_precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    class_recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    class_f1 = 2 * (class_precision * class_recall) / (class_precision + class_recall) if (class_precision + class_recall) > 0 else 0
    
    class_wise_precision.append(class_precision)
    class_wise_recall.append(class_recall)
    class_wise_f1.append(class_f1)

# Print the table with metrics for each class
print(f"{'Class':<15}{'Accuracy':<12}{'Precision':<12}{'Recall':<12}{'F1-Score':<12}")
print('-' * 60)
for i in range(len(classes)):
    print(f"{classes[i]:<15}{class_wise_accuracy[i]:<12.4f}{class_wise_precision[i]:<12.4f}{class_wise_recall[i]:<12.4f}{class_wise_f1[i]:<12.4f}")


In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Define the path to the new dataset directory
data_dir = 'path to dataset folder'

# Load the dataset from the directory
# Assuming the directory has subdirectories representing each class
def load_dataset(data_dir, img_size=(224, 224)):
    class_names = os.listdir(data_dir)  # List of class names
    images = []
    labels = []
    
    # Iterate through each class directory
    for class_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)  # Read image
            img = cv2.resize(img, img_size)  # Resize image
            img = img.astype(np.float32) / 255.0  # Normalize to [0, 1]
            images.append(img)
            labels.append(class_idx)  # Assign class index as label

    # Convert to numpy arrays
    images = np.array(images)
    labels = np.array(labels)
    
    # Shuffle the dataset
    indices = np.arange(images.shape[0])
    np.random.shuffle(indices)
    images = images[indices]
    labels = labels[indices]

    return images, labels, class_names

# Load the dataset
x_data, y_data, classes = load_dataset(data_dir)

# Split the data into training, validation, and test sets (e.g., 80-10-10 split)
train_split = int(0.8 * len(x_data))
val_split = int(0.10 * len(x_data))
x_train, y_train = x_data[:train_split], y_data[:train_split]
x_val, y_val = x_data[train_split:train_split + val_split], y_data[train_split:train_split + val_split]
x_test, y_test = x_data[train_split + val_split:], y_data[train_split + val_split:]

# Load the saved Siamese model
model_path = '/content/siamese_ecg_prod/siamese_cnn/model1.keras'
siamese_model = tf.keras.models.load_model(model_path)

# Predict on the test set
y_pred_proba = siamese_model.predict([x_test, x_test])
y_pred_classes = np.argmax(y_pred_proba, axis=1)

# Compute overall metrics
overall_accuracy = accuracy_score(y_test, y_pred_classes)
overall_f1 = f1_score(y_test, y_pred_classes, average='weighted')
overall_precision = precision_score(y_test, y_pred_classes, average='weighted')
overall_recall = recall_score(y_test, y_pred_classes, average='weighted')

# Print overall metrics
print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print(f"Overall F1-Score: {overall_f1:.4f}\n")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Initialize lists to store metrics for each class
class_wise_accuracy = []
class_wise_f1 = []
class_wise_precision = []
class_wise_recall = []

# Calculate metrics for each class
for i in range(len(classes)):
    true_positives = conf_matrix[i, i]
    false_positives = np.sum(conf_matrix[:, i]) - true_positives
    false_negatives = np.sum(conf_matrix[i, :]) - true_positives
    true_negatives = np.sum(conf_matrix) - (true_positives + false_positives + false_negatives)
    
    # Class-wise accuracy
    class_accuracy = (true_positives + true_negatives) / np.sum(conf_matrix)
    class_wise_accuracy.append(class_accuracy)
    
    # Class-wise precision, recall, and F1 score
    class_precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    class_recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    class_f1 = 2 * (class_precision * class_recall) / (class_precision + class_recall) if (class_precision + class_recall) > 0 else 0
    
    class_wise_precision.append(class_precision)
    class_wise_recall.append(class_recall)
    class_wise_f1.append(class_f1)

# Print the table with metrics for each class
print(f"{'Class':<15}{'Accuracy':<12}{'Precision':<12}{'Recall':<12}{'F1-Score':<12}")
print('-' * 60)
for i in range(len(classes)):
    print(f"{classes[i]:<15}{class_wise_accuracy[i]:<12.4f}{class_wise_precision[i]:<12.4f}{class_wise_recall[i]:<12.4f}{class_wise_f1[i]:<12.4f}")


In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt

# Define the path to the new dataset directory
data_dir = 'path/to/dataset/folder'

# Load the dataset from the directory
def load_dataset(data_dir, img_size=(224, 224)):
    class_names = os.listdir(data_dir)  # List of class names
    images = []
    labels = []
    
    # Iterate through each class directory
    for class_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)  # Read image
            img = cv2.resize(img, img_size)  # Resize image
            img = img.astype(np.float32) / 255.0  # Normalize to [0, 1]
            images.append(img)
            labels.append(class_idx)  # Assign class index as label

    # Convert to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels, class_names

# Load the dataset
x_data, y_data, classes = load_dataset(data_dir)

# Initialize KFold with k=7
kf = KFold(n_splits=7, shuffle=True)

# Load the saved Siamese model
model_path = 'path to saved model'

# Lists to store the overall metrics for each fold
fold_accuracies = []
fold_precisions = []
fold_recalls = []
fold_f1_scores = []

# Initialize lists for ROC curve and AUC
all_fpr = []
all_tpr = []
all_auc = []

# Loop over each fold
fold_num = 1
for train_index, test_index in kf.split(x_data):
    print(f"Fold {fold_num}:")

    # Split the data into training and test sets for this fold
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    # Load the model fresh for each fold (to ensure independent training)
    siamese_model = tf.keras.models.load_model(model_path)

    # Predict on the test set
    y_pred_proba = siamese_model.predict([x_test, x_test])
    y_pred_classes = np.argmax(y_pred_proba, axis=1)

    # Compute overall metrics
    overall_accuracy = accuracy_score(y_test, y_pred_classes)
    overall_f1 = f1_score(y_test, y_pred_classes, average='weighted')
    overall_precision = precision_score(y_test, y_pred_classes, average='weighted')
    overall_recall = recall_score(y_test, y_pred_classes, average='weighted')

    # Print overall metrics
    print(f"Accuracy: {overall_accuracy:.4f}")
    print(f"Precision: {overall_precision:.4f}")
    print(f"Recall: {overall_recall:.4f}")
    print(f"F1-Score: {overall_f1:.4f}\n")

    # Store the metrics for this fold
    fold_accuracies.append(overall_accuracy)
    fold_precisions.append(overall_precision)
    fold_recalls.append(overall_recall)
    fold_f1_scores.append(overall_f1)

    # Compute ROC curve and AUC for the fold
    y_test_bin = tf.keras.utils.to_categorical(y_test, num_classes=len(classes))
    fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_pred_proba.ravel())
    roc_auc = auc(fpr, tpr)

    # Store ROC curve data
    all_fpr.append(fpr)
    all_tpr.append(tpr)
    all_auc.append(roc_auc)

    fold_num += 1

# Plot ROC curve
plt.figure(figsize=(10, 8))

for i in range(len(all_fpr)):
    plt.plot(all_fpr[i], all_tpr[i], label=f'ROC Fold {i + 1} (AUC = {all_auc[i]:.4f})')

# Plot formatting
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line (random performance)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves for each fold')
plt.legend(loc="lower right")
plt.savefig('kcross_roc.png', dpi=600)
plt.show()

# Calculate the average metrics across all folds
avg_accuracy = np.mean(fold_accuracies)
avg_precision = np.mean(fold_precisions)
avg_recall = np.mean(fold_recalls)
avg_f1_score = np.mean(fold_f1_scores)

# Print average metrics across all folds
print("Average Metrics across all folds:")
print(f"Accuracy: {avg_accuracy:.4f}")
print(f"Precision: {avg_precision:.4f}")
print(f"Recall: {avg_recall:.4f}")
print(f"F1-Score: {avg_f1_score:.4f}")


In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt

# Load the saved Siamese model
model_path = 'path to saved model'
siamese_model = tf.keras.models.load_model(model_path)

# Predict on the entire dataset
y_pred_proba = siamese_model.predict([x_data, x_data])
y_pred_classes = np.argmax(y_pred_proba, axis=1)

# Convert labels to one-hot encoding
y_true_bin = tf.keras.utils.to_categorical(y_data, num_classes=len(classes))

# Compute Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_true_bin.ravel(), y_pred_proba.ravel())
avg_precision = average_precision_score(y_true_bin, y_pred_proba, average="weighted")

# Plot Precision-Recall curve
plt.figure(figsize=(10, 8))
plt.plot(recall, precision, label=f'Precision-Recall curve (AP = {avg_precision:.4f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.savefig('overall_pr_curve.png', dpi=650)
plt.show()

# Print Average Precision
print(f'Average Precision: {avg_precision:.4f}')
